<a href="https://colab.research.google.com/github/QiaoLin22/MASTER-LLM-DL/blob/main/LLM_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
first_tools = [
        # Tool 1 - Get Exchange Rate
        {
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the current exchange rate of a base currency and target currency",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_currency": {
                            "type": "string",
                            "description": "The base currency for exchange rate calculations, i.e. USD, EUR, RUB",
                        },
                        "target_currency": {
                            "type": "string",
                            "description": "The target currency for exchange rate calculations, i.e. USD, EUR, RUB"
                        },
                        "date": {
                            "type": "string",
                            "description": "A specific day to reference, in YYYY-MM-DD format."
                        },
                    },
                    "required": ["base_currency", "target_currency"],
                },
            },
        },
        # Tool 2 - Search Internet
        {
            "type": "function",
            "function": {
                "name": "search_internet",
                "description": "Get internet search results for real time information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The query to search the web for",
                        }
                    },
                    "required": ["search_query"],
                },
            },
        }
    ]

In [32]:
from openai import OpenAI
from getpass import getpass
import os
from google.colab import userdata

if os.getenv("OPENAI_API_KEY") is None:
  try:
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
  # if this notebook is being run outside of the Colab UI or you just don't have the necessary secret set
  except (userdata.TimeoutException, userdata.SecretNotFoundError):
    if any(['VSCODE' in x for x in os.environ.keys()]):
      print('Please enter password in the VS Code prompt at the top of your VS Code window!')
    os.environ["OPENAI_API_KEY"] = getpass("")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")
client = OpenAI()

messages = [{"role": "user",
             "content": "How much is a dollar worth in Japan? How about poland? Whats the current news in Argentina?"}]


response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=first_tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

OpenAI API key configured


In [9]:
# Function for printing out responses neatly
def pprint_response(response):
    print("--- Full Response ---\n")
    print(response, "\n")

    print("--- Chat Completion Message ---\n")
    print(response.choices[0].message, "\n")

    if response.choices[0].message.tool_calls:
        for i in range(0, len(response.choices[0].message.tool_calls)):
            print(f"--- Tool Call {i+1} ---\n")
            print(f"Function: {response.choices[0].message.tool_calls[i].function.name}\n")
            print(f"Arguments: {response.choices[0].message.tool_calls[i].function.arguments}\n")

In [10]:
pprint_response(response)

--- Full Response ---

ChatCompletion(id='chatcmpl-B9mSRaKzGtzWXeJqhIa8NBfLDE9Ab', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8IXyrkF9oMujKhm1g0JgCQic', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_AqNkUjg9QMP9grLqtICsidOn', function=Function(arguments='{"base_currency": "USD", "target_currency": "PLN"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_SuYaAjVRB61yiFddlKgnvILs', function=Function(arguments='{"search_query": "current news in Argentina"}', name='search_internet'), type='function')]))], created=1741670923, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_f9f4fb6dbf', usage=CompletionUsag

In [14]:
!pip install duckduckgo-search
from duckduckgo_search import DDGS
import requests

def search_internet(search_query: str) -> list:
    results = DDGS().text(str(search_query), max_results=5)
    return results

def get_exchange_rate(base_currency: str, target_currency: str, date: str = "latest") -> float:

    url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@{date}/v1/currencies/{base_currency.lower()}.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data.get(base_currency.lower(), {}).get(target_currency.lower(), None)
    else:
        raise Exception(f"Failed to fetch exchange rate: {response.status_code}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.3 MB/s eta 0:00:00


In [15]:
import inspect

# Main conversation function
def run_conversation(prompt, tools, tool_choice = "auto"):

    messages = [{"role": "user", "content": prompt}]

    print("\nInitial Message: ", messages)

    # Send the conversation and available functions to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )
    response_message = response.choices[0].message
    print("\nResponse Message: ", response_message)

    tool_calls = response_message.tool_calls
    print("\nTool Calls: ", tool_calls)

    # Check if the model wanted to call a function
    if tool_calls:

        # Call the functions
        available_functions = {
            "get_exchange_rate": get_exchange_rate,
            "search_internet": search_internet,
        }
        # extend conversation with assistant's reply
        messages.append(response_message)

        # Call the function and add the response
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)


            # Get the function signature and call the function with given arguments
            sig = inspect.signature(function_to_call)
            call_args = {
                k: function_args.get(k, v.default)
                for k, v in sig.parameters.items()
                if k in function_args or v.default is not inspect.Parameter.empty
            }
            print(f"\nCalling {function_to_call} with arguments {call_args}")

            function_response = str(function_to_call(**call_args))

            print("\nFunction Response: ", function_response)

            # Put output into a tool message
            tool_message = {
                    "tool_call_id": tool_call.id, # Needed for Parallel Tool Calling
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            print("\nAppending Message: ", tool_message)

            # Extend conversation with function response
            messages.append(tool_message)

        # Get a new response from the model where it can see the entire conversation including the function call outputs
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )

        print("\nLLM Response: ", second_response)

        print("\n---Formatted LLM Response---")
        print("\n",second_response.choices[0].message.content)

        return

prompt = "How much is a dollar worth in Japan? How about poland? Whats the current news in Argentina?"

run_conversation(prompt, first_tools)


Initial Message:  [{'role': 'user', 'content': 'How much is a dollar worth in Japan? How about poland? Whats the current news in Argentina?'}]

Response Message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_JAB7NosvXp0kFllkfyqHLV4E', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_FL4PGIYDNo3lYNxQeHz2eM9O', function=Function(arguments='{"base_currency": "USD", "target_currency": "PLN"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_932S3oQmKWmyb7xTRuehg2b6', function=Function(arguments='{"search_query": "current news in Argentina"}', name='search_internet'), type='function')])

Tool Calls:  [ChatCompletionMessageToolCall(id='call_JAB7NosvXp0kFllkfyqHLV4E', function=Function(arguments='{"base_currency": "USD", "target_currency

In [16]:
from pydantic import BaseModel, Field, HttpUrl
from typing import List, Optional, Dict
from datetime import datetime

# Defining the tool properties with Pydantic
class SendEmailCampaign(BaseModel):
    recipients: List[str] = Field(
        ...,
        description="List of strings, each an email address. Example: ['example1@mail.com', 'example2@mail.com']"
    )
    subject: str = Field(
        ...,
        description="String specifying the email's subject line. Example: 'Exciting News!'"
    )
    body_text: str = Field(
        ...,
        description="Plain text content of the email body. Example: 'We have some exciting updates to share with you.'"
    )
    attachments: Optional[List[HttpUrl]] = Field(
        default=[],
        description="List of URLs to attachment files. Example: ['http://example.com/attachment1.pdf', 'http://example.com/attachment2.png']"
    )
    personalization: Optional[Dict[str, Dict[str, str]]] = Field(
        default={},
        description="Dictionary for personalizing email content. Key is recipient email, value is a dictionary of variables. Example: {'example1@mail.com': {'first_name': 'John'}, 'example2@mail.com': {'first_name': 'Jane'}}"
    )
    send_time: Optional[datetime] = Field(
        None,
        description="The time when the email campaign is to be sent. Example: '2024-07-13T14:30:00'"
    )
    priority: Optional[str] = Field(
        default="normal",
        description="Email priority level. Options: 'low', 'normal', 'high'. Example: 'high'",
        enum=["low", "normal", "high"]
    )
    tracking: Optional[Dict[str, bool]] = Field(
        default={"open": True, "click": True},
        description="Tracking options for the email. Keys can be 'open' and 'click', values are booleans. Example: {'open': True, 'click': True}"
    )
    campaign_id: Optional[str] = Field(
        None,
        description="Unique identifier for the email campaign for tracking purposes. Example: 'campaign_12345'"
    )

# Creating the Tool with PyDantic Schema
email_campaign_tool = [
    {
        "type": "function",
        "function": {
            "name": "send_email_campaign",
            "description": "Send out a marketing email campaign",
            "parameters": SendEmailCampaign.schema(), # Passing in our Pydantic schema
        },
    }
]

<ipython-input-16-b9e0cf73d63b>:52: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  "parameters": SendEmailCampaign.schema(), # Passing in our Pydantic schema


In [17]:
email_campaign_tool

[{'type': 'function',
  'function': {'name': 'send_email_campaign',
   'description': 'Send out a marketing email campaign',
   'parameters': {'properties': {'recipients': {'description': "List of strings, each an email address. Example: ['example1@mail.com', 'example2@mail.com']",
      'items': {'type': 'string'},
      'title': 'Recipients',
      'type': 'array'},
     'subject': {'description': "String specifying the email's subject line. Example: 'Exciting News!'",
      'title': 'Subject',
      'type': 'string'},
     'body_text': {'description': "Plain text content of the email body. Example: 'We have some exciting updates to share with you.'",
      'title': 'Body Text',
      'type': 'string'},
     'attachments': {'anyOf': [{'items': {'format': 'uri',
         'maxLength': 2083,
         'minLength': 1,
         'type': 'string'},
        'type': 'array'},
       {'type': 'null'}],
      'default': [],
      'description': "List of URLs to attachment files. Example: ['http:

In [18]:
prompt = """
"Can you send an email to linqiao9797@gmail.com about the cool github page https://github.com/QiaoLin22/MASTER-LLM-DL/tree/main
for March 12th at 8 pm jst, this is high priority and include just click tracking and name personalization,
the campaign is campaign_13,
also add my linkedin as an attachment https://www.linkedin.com/in/qiao-lin-625511170/,
Sign off your response with Qiao Ł in the body.
"""

messages = [{"role": "user",
             "content": prompt}]


response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=email_campaign_tool,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

In [19]:
pprint_response(response)

--- Full Response ---

ChatCompletion(id='chatcmpl-B9mdpGvQyER63nlVzjH05sQtsddVy', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_myoNSfjYznIORd6F9fzqZIbp', function=Function(arguments='{"recipients":["linqiao9797@gmail.com"],"subject":"Check Out This Cool GitHub Page!","body_text":"Hi there,\\n\\nI hope you\'re doing well! I wanted to share this amazing GitHub page with you: https://github.com/QiaoLin22/MASTER-LLM-DL/tree/main. Let me know what you think!\\n\\nBest regards,\\nQiao Ł","attachments":["https://www.linkedin.com/in/qiao-lin-625511170/"],"personalization":{"linqiao9797@gmail.com":{"first_name":"Lin Qiao"}},"send_time":"2024-03-12T20:00:00+09:00","priority":"high","tracking":{"click":true},"campaign_id":"campaign_13"}', name='send_email_campaign'), type='function')]))], created=1741671629, model='g

In [20]:
# Example function of sending off an Email Campaign
def send_email_campaign(data: dict):
    try:
        # Validate and parse the data using the SendEmailCampaign model
        campaign = SendEmailCampaign(**data)

        # Simulate sending the email campaign
        print(f"\nSending email campaign '{campaign.campaign_id or 'N/A'}' with priority '{campaign.priority}':")
        for recipient in campaign.recipients:
            personalized_subject = campaign.subject
            if recipient in campaign.personalization:
                if 'first_name' in campaign.personalization[recipient]:
                    personalized_subject = f"{campaign.personalization[recipient]['first_name']}, {campaign.subject}"

            print("-----")
            print(f"\nTo: {recipient}")
            print(f"Subject: {personalized_subject}")
            print(f"Body: {campaign.body_text}")
            if campaign.attachments:
                attachment_urls = [str(url) for url in campaign.attachments]
                print(f"\tAttachments: {', '.join(attachment_urls)}")

        print("-----")
        print(f"\nScheduled Send Time: {campaign.send_time or 'Immediate'}")
        print(f"Campaign ID: {campaign.campaign_id or 'N/A'}")
        print(f"Tracking: {campaign.tracking}")
    except Exception as e:
        print(f"Failed to send email campaign: {e}")

args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
send_email_campaign(args)


Sending email campaign 'campaign_13' with priority 'high':
-----

To: linqiao9797@gmail.com
Subject: Lin Qiao, Check Out This Cool GitHub Page!
Body: Hi there,

I hope you're doing well! I wanted to share this amazing GitHub page with you: https://github.com/QiaoLin22/MASTER-LLM-DL/tree/main. Let me know what you think!

Best regards,
Qiao Ł
	Attachments: https://www.linkedin.com/in/qiao-lin-625511170/
-----

Scheduled Send Time: 2024-03-12 20:00:00+09:00
Campaign ID: campaign_13
Tracking: {'click': True}


In [22]:
!pip install langchain_anthropic
from langchain_anthropic import ChatAnthropic

# Claude 3.5 Sonnet
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.7)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 6.5 MB/s eta 0:00:00


Defining Tools
LangChain offers multiple ways of defining tools, you can:

Use the @tool decorator along with a description on an exisiting function
Use Pydantic schemas to define tool arguments
Use built in tools as part of LangChain's integrations
Use a combination of all above methods to define custom tools from LangChain's BaseTool class

In [24]:
from langchain_core.tools import tool

# Using the tool decorator to attach onto an existing function
@tool
def get_exchange_rate(base_currency: str, target_currency: str, date: str = "latest") -> float:
    """Get the latest exchange rate between two currency. Date defaults latest if not provided."""
    url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{base_currency.lower()}.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data.get(base_currency.lower(), {}).get(target_currency.lower(), None)
    else:
        raise Exception(f"Failed to fetch exchange rate: {response.status_code}")


# Using Pydantic schemas (non-invokable tool)
from langchain_core.pydantic_v1 import BaseModel, Field

class SendEmailCampaignLC(BaseModel):
    "Send a marketing email campaign out to your mailing list"

    recipients: List[str] = Field(
        ...,
        description="List of strings, each an email address. Example: ['example1@mail.com', 'example2@mail.com']"
    )
    subject: str = Field(
        ...,
        description="String specifying the email's subject line. Example: 'Exciting News!'"
    )
    body_text: str = Field(
        ...,
        description="Plain text content of the email body. Example: 'We have some exciting updates to share with you.'"
    )

# Using built in LangChain tool integrations
!pip install langchain_community
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(max_results=10)
web_search = DuckDuckGoSearchResults(api_wrapper=wrapper)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [25]:
get_exchange_rate.invoke({"base_currency": "usd", "target_currency": "jpy"})

146.83516104

In [26]:
web_search.invoke("Fun things to do in SF")

"snippet: San Francisco is only 7 miles square, but it's packed with activities to delight outdoorsy types, art and culture lovers, foodies and curious wanderers of all ages., title: 30 Top-Rated Things to Do in San Francisco | U.S. News Travel, link: https://travel.usnews.com/San_Francisco_CA/Things_To_Do/, snippet: Here are the 52 best things to do in San Francisco. From beautiful hikes to rooftop bars with spectacular views, San Francisco's a joy to explore. Tuesday November 12 2024. Share. Copy Link., title: Here are the 52 best things to do in San Francisco - Time Out, link: https://www.timeout.com/san-francisco/things-to-do/best-things-to-do-in-san-francisco, snippet: The audio tour is one of the best things to do in San Francisco for tourists, but even locals will have plenty to learn on a visit to this historic island. 23. Ride a cable car. SF's cable cars are the only moving national landmark in the United States, dating back to 1873., title: 50 Essential Activities For Your 2

In [27]:
tools = [web_search, SendEmailCampaignLC, get_exchange_rate]

llm_tools = llm.bind_tools(tools)

In [28]:
# prompt = "How much is a dollar worth in Japan right now"
# prompt = "Can you send an email to adamlucek@youtube.com, samaltman@openai.com, and elonmusk@twitter.com about the cool youtube channel https://www.youtube.com/@AdamLucek"
# prompt = "When was langgraph cloud released?"
prompt = "When was langchain founded? and how much is a dollar worth in japan rn"

output = llm_tools.invoke(prompt)
print("\n---Response---")
print(output.content[0]['text'])
print("\n---Tool Calls---")
print(output.tool_calls)

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

#Handling Tool Calls
Execute and feed back the tools once we got the arguments from the LLM, now that the tools and messages are formatted by LangChain

Tools can be directly invoked when formatted as a LangChain tool, simplifying the execution process greatly

In [29]:
from langchain_core.messages import HumanMessage, ToolMessage

query = "When was langchain founded? and how much is a dollar worth in japan rn"

messages = [HumanMessage(query)]
ai_msg = llm_tools.invoke(messages)
messages.append(ai_msg)

# Only using web search and exchange rate, and the Pydantic schema is not a full function, just a container for arguments
for tool_call in ai_msg.tool_calls:
    selected_tool = {"duckduckgo_results_json": web_search, "get_exchange_rate": get_exchange_rate}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

for i in range(0, len(messages)):
    print("-------")
    print(f"{messages[i].type}: ", messages[i])

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

In [30]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Get the prompt
oaif_prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.7)
tools = [web_search, get_exchange_rate]

# Create the agent
oaif_agent = create_tool_calling_agent(llm, tools, oaif_prompt)

# Create the Agent Executor
# This is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats. I
oaif_agent_executor = AgentExecutor(agent=oaif_agent, tools=tools, verbose=True)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [33]:
query = "When was langgraph cloud released? and how much is a dollar worth in japan rn"

response = oaif_agent_executor.invoke({"input": query})

print("\n", response['output'][0]['text'])



> Entering new AgentExecutor chain...


TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"